In [ ]:
import pandas as pd
import altair as alt
import utils

IN_FILE = "data/single_mora_words.csv"


In [ ]:
words = pd.read_csv(IN_FILE)[['word_id', 'sem_word_id', 'midashigo_alt', 'accent']]

In [ ]:
alt.Chart(
    words.melt(id_vars=['word_id', 'sem_word_id', 'midashigo_alt'])
).mark_bar().encode(
    x='midashigo_alt:N',
    y='count()',
    color='value:N',
)


In [ ]:
words = utils.get_last_kana_info(words)
words 

In [ ]:
def make_chart(df, group):
    word_count = (df.groupby([group, 'accent']).word_id.count().reset_index().rename(columns={'word_id':'word_count'}).set_index(group))
    word_count['total'] = word_count.groupby(group).word_count.sum()
    word_count['density'] = word_count.word_count / word_count.total
    word_count = word_count.reset_index()
    # return word_count.groupby(group).first()
    return alt.Chart(word_count).mark_bar().encode(
        x=group,
        y='density:Q',
        color='accent:N',
    ).properties(
        width=300
    ) + alt.Chart(word_count.groupby(group).first().reset_index()).mark_text(dy=140, color='white').encode(
        x=group,
        text=alt.Text('total:Q')
    )

# make_chart(words, 'end_vowel')
make_chart(words, 'is_vowel') | make_chart(words, 'end_vowel') | make_chart(words, 'onset')